# Analisis Exploratorio de Datos (EDA)
## Aplicacion de Machine Learning para la Prediccion de Desempleo Regional en Panama

Este notebook presenta un analisis exploratorio completo del dataset de desempleo
por provincia en Panama, cubriendo:

1. **Vision general del dataset** - Estructura, dimensiones, tipos de datos
2. **Valores faltantes** - Patron de nulos por variable y periodo
3. **Distribuciones** - Histogramas y estadisticas descriptivas
4. **Tendencias temporales** - Evolucion del desempleo 2011-2024
5. **Analisis por provincia** - Comparaciones regionales
6. **Impacto del COVID-19** - Cambio estructural 2020-2021
7. **Correlaciones** - Relaciones entre variables
8. **Brechas** - Genero y urbano/rural

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import sys
from pathlib import Path

# Configuracion
ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(ROOT))
from src.config import PROCESSED_DATA_DIR, RIESGO_BAJO, RIESGO_CRITICO, COLOR_MAP

sns.set_theme(style='whitegrid', palette='muted', font_scale=1.1)
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print(f'Ruta del proyecto: {ROOT}')

## 1. Vision General del Dataset

In [ ]:
df = pd.read_csv(PROCESSED_DATA_DIR / 'desempleo_por_provincia.csv')
print(f'Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas')
print(f'\nProvincias ({df["provincia"].nunique()}): {sorted(df["provincia"].unique())}')
print(f'\nPeriodos ({df["periodo"].nunique()}): {sorted(df["periodo"].unique())}')
print(f'\nAreas: {df["area"].unique().tolist() if "area" in df.columns else "N/A"}')
print(f'Sexos: {df["sexo"].unique().tolist() if "sexo" in df.columns else "N/A"}')

In [ ]:
# Tipos de datos y memoria
print('Tipos de datos:')
print(df.dtypes.value_counts())
print(f'\nMemoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')
print(f'\n--- Primeras 5 filas ---')
df.head()

In [ ]:
# Estadisticas descriptivas de las variables numericas principales
cols_principales = ['tasa_desempleo', 'tasa_participacion', 'pea', 'ocupados',
                    'desocupados', 'pct_subempleo', 'empleo_informal_pct',
                    'mediana_salario', 'pib_crecimiento']
cols_disponibles = [c for c in cols_principales if c in df.columns]
df[cols_disponibles].describe().round(2)

## 2. Valores Faltantes

In [ ]:
# Porcentaje de nulos por columna
nulos = df.isnull().sum()
nulos_pct = (nulos / len(df) * 100).round(1)
nulos_df = pd.DataFrame({'Nulos': nulos, '%': nulos_pct})
nulos_df = nulos_df[nulos_df['Nulos'] > 0].sort_values('%', ascending=False)

if len(nulos_df) > 0:
    print(f'Columnas con valores faltantes: {len(nulos_df)}')
    print(f'Total de celdas nulas: {nulos.sum()} ({nulos.sum()/(df.shape[0]*df.shape[1])*100:.1f}%)')
    display(nulos_df)
    
    # Heatmap de nulos por periodo y provincia (area=total, sexo=total)
    df_total = df[(df['area']=='total') & (df['sexo']=='total')].copy()
    cols_num = df_total.select_dtypes(include=[np.number]).columns[:15]
    nulos_pivot = df_total.pivot_table(
        index='provincia', columns='periodo',
        values='tasa_desempleo', aggfunc=lambda x: x.isna().sum()
    )
    fig, ax = plt.subplots(figsize=(14, 6))
    sns.heatmap(nulos_pivot, cmap='YlOrRd', annot=True, fmt='g', ax=ax)
    ax.set_title('Valores faltantes en tasa_desempleo por Provincia y Periodo')
    plt.tight_layout()
    plt.show()
else:
    print('No hay valores faltantes en el dataset.')

## 3. Distribuciones

In [ ]:
# Distribucion de la tasa de desempleo (variable objetivo)
df_total = df[(df['area']=='total') & (df['sexo']=='total')].copy()

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Histograma
axes[0].hist(df_total['tasa_desempleo'].dropna(), bins=25, color='steelblue', edgecolor='white')
axes[0].axvline(RIESGO_BAJO, color=COLOR_MAP['moderado'], ls='--', label=f'Umbral bajo ({RIESGO_BAJO}%)')
axes[0].axvline(RIESGO_CRITICO, color=COLOR_MAP['critico'], ls='--', label=f'Umbral critico ({RIESGO_CRITICO}%)')
axes[0].set_xlabel('Tasa de Desempleo (%)')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribucion de Tasa de Desempleo')
axes[0].legend(fontsize=8)

# Boxplot por provincia
orden = df_total.groupby('provincia')['tasa_desempleo'].median().sort_values(ascending=False).index
sns.boxplot(data=df_total, y='provincia', x='tasa_desempleo', order=orden, ax=axes[1],
            palette='RdYlGn_r')
axes[1].set_title('Desempleo por Provincia')
axes[1].set_xlabel('Tasa de Desempleo (%)')

# QQ-plot
from scipy import stats
stats.probplot(df_total['tasa_desempleo'].dropna(), dist='norm', plot=axes[2])
axes[2].set_title('Q-Q Plot (Normalidad)')

plt.tight_layout()
plt.show()

print(f'\nTest de normalidad (Shapiro-Wilk):')
stat, p = stats.shapiro(df_total['tasa_desempleo'].dropna().sample(min(500, len(df_total))))
print(f'  W={stat:.4f}, p={p:.4e} -> {"No normal" if p < 0.05 else "Normal"} (alpha=0.05)')

In [ ]:
# Distribuciones de variables principales
vars_plot = ['tasa_participacion', 'pct_subempleo', 'empleo_informal_pct',
             'mediana_salario', 'pct_universitaria', 'pct_sector_terciario']
vars_plot = [v for v in vars_plot if v in df_total.columns]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
for i, var in enumerate(vars_plot):
    ax = axes[i//3, i%3]
    sns.histplot(df_total[var].dropna(), kde=True, ax=ax, color='steelblue')
    ax.set_title(var)
    ax.set_xlabel('')

plt.suptitle('Distribuciones de Variables Principales', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 4. Tendencias Temporales

In [ ]:
# Evolucion temporal del desempleo por provincia
df_total = df[(df['area']=='total') & (df['sexo']=='total')].copy()
df_total = df_total.sort_values('periodo')

fig = px.line(
    df_total, x='periodo', y='tasa_desempleo',
    color='provincia', markers=True,
    title='Evolucion del Desempleo por Provincia (2011-2024)',
    labels={'periodo': 'Periodo', 'tasa_desempleo': 'Tasa de Desempleo (%)', 'provincia': 'Provincia'}
)
fig.add_hline(y=RIESGO_BAJO, line_dash='dash', line_color='orange',
              annotation_text=f'Umbral bajo ({RIESGO_BAJO}%)')
fig.add_hline(y=RIESGO_CRITICO, line_dash='dash', line_color='red',
              annotation_text=f'Umbral critico ({RIESGO_CRITICO}%)')
fig.update_layout(height=500, template='plotly_white',
                  legend=dict(orientation='h', y=-0.3))
fig.show()

In [ ]:
# Media nacional por periodo
media_nacional = df_total.groupby('periodo')['tasa_desempleo'].agg(['mean', 'std', 'min', 'max']).reset_index()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(media_nacional['periodo'], media_nacional['mean'], 'o-', color='steelblue', lw=2, label='Media')
ax.fill_between(media_nacional['periodo'],
                media_nacional['mean'] - media_nacional['std'],
                media_nacional['mean'] + media_nacional['std'],
                alpha=0.2, color='steelblue', label='+-1 std')
ax.plot(media_nacional['periodo'], media_nacional['min'], '--', color='green', alpha=0.6, label='Min')
ax.plot(media_nacional['periodo'], media_nacional['max'], '--', color='red', alpha=0.6, label='Max')
ax.axhline(RIESGO_BAJO, color='orange', ls=':', alpha=0.7)
ax.axhline(RIESGO_CRITICO, color='red', ls=':', alpha=0.7)
ax.set_xlabel('Periodo')
ax.set_ylabel('Tasa de Desempleo (%)')
ax.set_title('Tendencia Nacional del Desempleo (Media +/- 1 std)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Analisis por Provincia

In [ ]:
# Ranking de provincias por desempleo promedio
ranking = df_total.groupby('provincia')['tasa_desempleo'].agg(['mean', 'std', 'min', 'max']).round(2)
ranking = ranking.sort_values('mean', ascending=False)
ranking.columns = ['Media', 'Desv. Std.', 'Minimo', 'Maximo']
print('Ranking de Provincias por Desempleo Promedio:')
display(ranking)

In [ ]:
# Heatmap interactivo: provincia x periodo
pivot = df_total.pivot_table(index='provincia', columns='periodo',
                              values='tasa_desempleo')
pivot = pivot.reindex(ranking.index)  # Ordenar por media descendente

fig = px.imshow(
    pivot, text_auto='.1f',
    color_continuous_scale='RdYlGn_r',
    title='Tasa de Desempleo por Provincia y Periodo',
    labels=dict(x='Periodo', y='Provincia', color='Tasa (%)'),
    aspect='auto'
)
fig.update_layout(height=500, template='plotly_white')
fig.show()

In [ ]:
# Violin plot: distribucion detallada por provincia
fig, ax = plt.subplots(figsize=(14, 7))
sns.violinplot(data=df_total, x='provincia', y='tasa_desempleo',
               order=ranking.index, palette='RdYlGn_r', inner='box', ax=ax)
ax.axhline(RIESGO_BAJO, color='orange', ls='--', alpha=0.7)
ax.axhline(RIESGO_CRITICO, color='red', ls='--', alpha=0.7)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_title('Distribucion del Desempleo por Provincia (Violin Plot)')
ax.set_xlabel('')
ax.set_ylabel('Tasa de Desempleo (%)')
plt.tight_layout()
plt.show()

## 6. Impacto del COVID-19

In [ ]:
# Comparar periodos pre-COVID vs post-COVID
df_total = df[(df['area']=='total') & (df['sexo']=='total')].copy()
df_total['etapa'] = df_total['anio'].apply(
    lambda x: 'Pre-COVID (2011-2019)' if x < 2020 else 'Post-COVID (2020-2024)'
)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot comparativo
sns.boxplot(data=df_total, x='etapa', y='tasa_desempleo', ax=axes[0], palette=['steelblue', 'coral'])
axes[0].set_title('Desempleo: Pre vs Post COVID')
axes[0].set_xlabel('')
axes[0].set_ylabel('Tasa de Desempleo (%)')

# Cambio por provincia
pre = df_total[df_total['etapa'].str.contains('Pre')].groupby('provincia')['tasa_desempleo'].mean()
post = df_total[df_total['etapa'].str.contains('Post')].groupby('provincia')['tasa_desempleo'].mean()
cambio = (post - pre).sort_values(ascending=True)

colores = ['coral' if v > 0 else 'steelblue' for v in cambio.values]
axes[1].barh(cambio.index, cambio.values, color=colores)
axes[1].axvline(0, color='black', lw=0.8)
axes[1].set_title('Cambio en Desempleo: Post-COVID vs Pre-COVID')
axes[1].set_xlabel('Cambio en puntos porcentuales')

plt.tight_layout()
plt.show()

print(f'\nMedia pre-COVID: {pre.mean():.2f}%')
print(f'Media post-COVID: {post.mean():.2f}%')
print(f'Cambio promedio: {(post.mean() - pre.mean()):+.2f} pp')

In [ ]:
# Test estadistico: diferencia significativa pre vs post COVID
from scipy.stats import mannwhitneyu

pre_vals = df_total[df_total['etapa'].str.contains('Pre')]['tasa_desempleo'].dropna()
post_vals = df_total[df_total['etapa'].str.contains('Post')]['tasa_desempleo'].dropna()

stat, p = mannwhitneyu(pre_vals, post_vals, alternative='two-sided')
print(f'Mann-Whitney U test: U={stat:.0f}, p={p:.4e}')
print(f'Diferencia significativa (alpha=0.05): {"Si" if p < 0.05 else "No"}')

## 7. Correlaciones

In [ ]:
# Matriz de correlacion de variables principales
df_total = df[(df['area']=='total') & (df['sexo']=='total')].copy()
cols_corr = ['tasa_desempleo', 'tasa_participacion', 'pct_subempleo',
             'empleo_informal_pct', 'pct_universitaria', 'pct_sin_educacion',
             'pct_sector_primario', 'pct_sector_terciario', 'pct_empresa_grande',
             'mediana_salario', 'pib_crecimiento']
cols_corr = [c for c in cols_corr if c in df_total.columns]

corr = df_total[cols_corr].corr()

fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, linewidths=0.5, ax=ax)
ax.set_title('Matriz de Correlacion - Variables Principales', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Top correlaciones con la variable objetivo
corr_target = corr['tasa_desempleo'].drop('tasa_desempleo').abs().sort_values(ascending=False)
print('Correlaciones con tasa_desempleo (valor absoluto):')
for var, val in corr_target.items():
    signo = '+' if corr['tasa_desempleo'][var] > 0 else '-'
    print(f'  {signo} {var:<30} {val:.3f}')

In [ ]:
# Scatter plots de las variables mas correlacionadas con desempleo
top_vars = corr_target.head(4).index.tolist()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for i, var in enumerate(top_vars):
    ax = axes[i//2, i%2]
    sns.scatterplot(data=df_total, x=var, y='tasa_desempleo',
                    hue='provincia', alpha=0.6, ax=ax, legend=False)
    # Linea de tendencia
    z = np.polyfit(df_total[var].dropna(), df_total.loc[df_total[var].notna(), 'tasa_desempleo'], 1)
    p_line = np.poly1d(z)
    x_range = np.linspace(df_total[var].min(), df_total[var].max(), 100)
    ax.plot(x_range, p_line(x_range), 'r--', alpha=0.8)
    r = corr['tasa_desempleo'][var]
    ax.set_title(f'{var} (r={r:.2f})')
    ax.set_ylabel('Tasa de Desempleo (%)')

plt.suptitle('Variables mas Correlacionadas con el Desempleo', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 8. Brechas de Genero y Urbano/Rural

In [ ]:
# Brecha de genero: mujeres vs hombres
df_genero = df[(df['area']=='total')].copy()

hombres = df_genero[df_genero['sexo']=='hombres'].groupby('periodo')['tasa_desempleo'].mean()
mujeres = df_genero[df_genero['sexo']=='mujeres'].groupby('periodo')['tasa_desempleo'].mean()
total = df_genero[df_genero['sexo']=='total'].groupby('periodo')['tasa_desempleo'].mean()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Evolucion por genero
axes[0].plot(hombres.index, hombres.values, 'o-', label='Hombres', color='steelblue')
axes[0].plot(mujeres.index, mujeres.values, 's-', label='Mujeres', color='coral')
axes[0].plot(total.index, total.values, '^--', label='Total', color='gray', alpha=0.7)
axes[0].set_title('Desempleo por Genero')
axes[0].set_ylabel('Tasa de Desempleo (%)')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)

# Brecha (mujeres - hombres)
brecha = mujeres - hombres
colores_brecha = ['coral' if v > 0 else 'steelblue' for v in brecha.values]
axes[1].bar(brecha.index, brecha.values, color=colores_brecha)
axes[1].axhline(0, color='black', lw=0.8)
axes[1].set_title('Brecha de Genero (Mujeres - Hombres)')
axes[1].set_ylabel('Diferencia (pp)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f'Brecha promedio (Mujeres - Hombres): {brecha.mean():+.2f} pp')

In [ ]:
# Brecha urbano/rural
df_area = df[(df['sexo']=='total')].copy()

urbana = df_area[df_area['area']=='urbana'].groupby('periodo')['tasa_desempleo'].mean()
rural = df_area[df_area['area']=='rural'].groupby('periodo')['tasa_desempleo'].mean()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(urbana.index, urbana.values, 'o-', label='Urbana', color='steelblue')
axes[0].plot(rural.index, rural.values, 's-', label='Rural', color='forestgreen')
axes[0].set_title('Desempleo: Urbano vs Rural')
axes[0].set_ylabel('Tasa de Desempleo (%)')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)

brecha_area = urbana - rural
colores_ba = ['coral' if v > 0 else 'forestgreen' for v in brecha_area.values]
axes[1].bar(brecha_area.index, brecha_area.values, color=colores_ba)
axes[1].axhline(0, color='black', lw=0.8)
axes[1].set_title('Brecha Urbano-Rural (Urbana - Rural)')
axes[1].set_ylabel('Diferencia (pp)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f'Brecha promedio (Urbana - Rural): {brecha_area.mean():+.2f} pp')

## 9. Outliers

In [ ]:
# Deteccion de outliers con IQR
df_total = df[(df['area']=='total') & (df['sexo']=='total')].copy()

Q1 = df_total['tasa_desempleo'].quantile(0.25)
Q3 = df_total['tasa_desempleo'].quantile(0.75)
IQR = Q3 - Q1
lim_inf = Q1 - 1.5 * IQR
lim_sup = Q3 + 1.5 * IQR

outliers = df_total[(df_total['tasa_desempleo'] < lim_inf) | (df_total['tasa_desempleo'] > lim_sup)]

print(f'IQR: {IQR:.2f}')
print(f'Limites: [{lim_inf:.2f}, {lim_sup:.2f}]')
print(f'Outliers encontrados: {len(outliers)} ({len(outliers)/len(df_total)*100:.1f}%)')

if len(outliers) > 0:
    print(f'\nOutliers (por encima de {lim_sup:.1f}%):')
    outliers_altos = outliers[outliers['tasa_desempleo'] > lim_sup]
    if len(outliers_altos) > 0:
        display(outliers_altos[['provincia', 'periodo', 'tasa_desempleo']].sort_values(
            'tasa_desempleo', ascending=False).head(10))

## 10. Conclusiones del EDA

### Hallazgos principales:

1. **Heterogeneidad regional**: Las comarcas indigenas y Colon presentan tasas de desempleo
   consistentemente mas altas que el resto del pais.

2. **Impacto COVID**: El periodo 2020-2021 muestra un incremento significativo del desempleo,
   con recuperacion parcial en 2023-2024.

3. **Brecha de genero**: Las mujeres presentan tasas de desempleo consistentemente superiores
   a los hombres en todos los periodos.

4. **Brecha urbano-rural**: El desempleo urbano tiende a ser mayor que el rural,
   lo que puede reflejar diferencias en la estructura economica.

5. **Variables correlacionadas**: La informalidad, el subempleo y el nivel educativo
   muestran las correlaciones mas fuertes con la tasa de desempleo.

6. **Distribucion no normal**: La tasa de desempleo tiene una distribucion asimetrica
   positiva (sesgo a la derecha), con outliers en las comarcas.

Estos hallazgos informan la seleccion de features y la interpretacion del modelo predictivo.